In [16]:

# # Classification

# ## Imports

In [17]:


import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import esm
import math

In [26]:

# ## augments


import argparse

# Create the parser
parser = argparse.ArgumentParser(description="Augments for embeding.")

# Add arguments
parser.add_argument('--model', 
                    type=int, 
                    help="index of model to use from ESM 0:5: %(default)", 
                    default=1)
parser.add_argument('--cc', 
                    type=int, 
                    help="Chain choice. 0: sep, 1: together %(default)", 
                    default=0)

parser.add_argument('--file', 
                    type=str, 
                    help="file to embed %(default)", 
                    default="anti")
parser.add_argument('--run_local', 
                    type=bool, 
                    help="to run local  %(default)", 
                    default=True)

# Parse the arguments
args = parser.parse_args()

# Access the arguments
# name = args.name
model_args = args.model
cc_args = args.cc
file_args = args.file

local_run = args.run_local




In [33]:




if local_run :
    print("using local seting")

    ##### model options
    # ESM-2 8M model     # 0 
    # ESM-2 35M model   # 1
    # ESM-2 150M model  # 2
    # ESM-2 650M model  # 3
    # ESM-2 3B model    # 4 
    # ESM-2 15B model   # 5
    model_args = 4


    ##### chain choce 
    # 0: sep, 1 together
    # cc_args = 1     
    cc_args = 0 

    # file choice
    # anti or cova as string
    # file_args = "anti"
    file_args = "cova" 
    print(f"___{file_args=}__\n__{model_args=}___\n___{cc_args=}___\n")



if model_args not in range(6):
    print("model index not valid")
    if cc_args not in range(2):
        print("cc choince not valid")
        if file_args not in ["anti","cova"]:
            print("lol")
            # sys.exit()






# model, alphabet = torch.hub.load("facebookresearch/esm:main", "esm2_t33_650M_UR50D")



# ## 

# ## Data Load

using local seting
___file_args='cova'__
__model_args=4___
___cc_args=0___



In [187]:

work_dir = os.getcwd()
data_dir = os.path.join(work_dir, '../data')

if file_args == "anti":
    print("label file in excel format")
    data = pd.read_excel(os.path.join(data_dir, 'external/antibody_info.xlsx'), header=1)
elif file_args == "cova":
    print("unlabel file in csv format")
    data = pd.read_csv(os.path.join(data_dir, 'external/covabdab_search_results.csv'))
else:
    print("work in prgoess:")




unlabel file in csv format


In [183]:

data.iloc[[96, 98, 101]]["VL"]

96     NaN
98     NaN
101    NaN
Name: VL, dtype: object

In [191]:
### filter
if file_args == "cova":
    print("filtering for VL chains 'nan' ")
    print("nan in these index")
    where_na = data[data['VL'].isna()].index
    print(where_na)

    print(f"there were drops this many rows: {len(where_na)}")
    data = data.dropna(subset=['VL'])





filtering for VL chains 'nan' 
nan in these index
Index([96, 98, 101, 182, 1290], dtype='int64')
there were drops this many rows: 5


In [192]:

#### # extacting file chains to embed

if file_args == "anti":
    print("geting chains in label")
    
    # get the name and chains
    chains = data[["Antibody  Name","Heavy chain AA","Light chain AA"]]



elif file_args == "cova":
    print("geting chains in unlabel")


    # get the name and chains
    chains = data[["Name","VHorVHH","VL"]]
    chains.columns = ["Antibody  Name","Heavy chain AA","Light chain AA"]

else:
    print("work in prgoess:")


# display(df.head())

geting chains in unlabel


In [193]:


#chain mode,
# Either seporate embeding or together by cls token as linker
# Seporate heavy chains is frst.

chain_choice = ["seperate","together"]
chain_mode = chain_choice[cc_args]
print(f"chain mode: {chain_mode} \n")

df = chains

# embed as septer
if cc_args == 0: 

    print("________seting data long/seperate________\n\n")


    # pivot longer
    df = pd.melt(df, id_vars="Antibody  Name", value_vars=["Heavy chain AA","Light chain AA"],var_name = "Chain ID",value_name = "chain AA")
    # new identy colun, combi of antibody and chain name
    df["identifyer"] = df["Antibody  Name"].astype(str) + " / " + df["Chain ID"]

    # subset to mine colummn
    df = df[["identifyer","chain AA"]]


else: 
    # embed together with cls as divider
    
    # Token to insert in the middle
    link_token = "<cls>"

    # concat wtih toek
    print(f"______seting data together with: {link_token}______\n\n")


    # Combine the columns with the token in between
    df['chain AA'] = df['Heavy chain AA'] + link_token + df['Light chain AA']

    # lazy code
    df["identifyer"] = df["Antibody  Name"]


    # subset to relevant colmun
    df = df[["identifyer","chain AA"]]


# format to esm, list of tubles with (name, seq) 
esm_input = list(zip(*map(df.get, df[["identifyer","chain AA"]])))



###
# display(esm_input[:2])



chain mode: seperate 

________seting data long/seperate________




In [ ]:

print("down/load model\n")
torch.hub.set_dir(data_dir)

esm2_model_names = [
    'esm2_t6_8M_UR50D',    # ESM-2 8M model     # 0 
    'esm2_t12_35M_UR50D',   # ESM-2 35M model   # 1
    'esm2_t30_150M_UR50D',  # ESM-2 150M model  # 2
    'esm2_t33_650M_UR50D',  # ESM-2 650M model  # 3
    'esm2_t36_3B_UR50D',    # ESM-2 3B model    # 4 
    'esm2_t48_15B_UR50D'    # ESM-2 15B model   # 5
]

model_name = esm2_model_names[model_args]
print(f"model to use: {model_name}")

model_path = data_dir+"/checkpoints/"+model_name+".pt"

if True == os.path.isfile(model_path):
    # local file
    print("model found  in local file")
    model, alphabet = esm.pretrained.load_model_and_alphabet(model_path)
else:
    # downloading
    print("download file")
    model, alphabet = esm.pretrained.load_model_and_alphabet(model_name)
    

# model, alphabet = esm.pretrained.esm2_t6_8M_UR50D()
# model, alphabet = esm.pretrained.esm2_t12_35M_UR50D()
# model, alphabet = esm.pretrained.esm2_t30_150M_UR50D()
# model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
# model, alphabet = esm.pretrained.esm2_t36_3B_UR50D()
# model, alphabet = esm.pretrained.esm2_t48_15B_UR50D()




batch_converter = alphabet.get_batch_converter()
model.eval()  # disables dropout for deterministic results




down/load model

model to use: esm2_t36_3B_UR50D
model found  in local file


/anaconda/envs/aipro/lib/python3.13/site-packages/esm/pretrained.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_data = torch.load(str(model_location), map_locati

KeyboardInterrupt: 

In [ ]:

batch_size = 10

# data format needs to follow
# data = [
#     ("protein1", "MKTVRQERLKSIVRILERSKEPVSGAQLAEELSVSRQVIVQDIAYLRSLGYNIVATPRGYVLAGG"),
#     ("protein2", "KALTARQQEVFDLIRDHISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein2 with mask","KALTARQQEVFDLIRD<mask>ISQTGMPPTRAEIAQRLGFRSPNAAEEHLKALARKGVIEIVSGASRGIRLLQEE"),
#     ("protein3",  "K A <mask> I S Q"),
# ]


# 
data_esm = esm_input#[0:10]  # for testing local



# get label name, string length and boken of sequence
batch_labels, batch_strs, batch_tokens = batch_converter(data_esm)


# used for sequence representaion.
# batch_lens = (batch_tokens != alphabet.padding_idx).sum(1)

last_layer= sum(1 for layer in model.modules() if "trans" in str(type(layer)).lower())
print("embeding")

# if torch.cuda.is_available():
#     batch_tokens = batch_tokens.cuda()
#     model = model.cuda()

all_embeddings = []
all_contacts = []


number_batches = math.ceil(len(df)/batch_size)

#local_test
# number_batches = 2

for batch_index in range(number_batches):
    if batch_index % 10 ==0:
        print(f"{(batch_index/number_batches)*100} % done")

    batch_to_run = batch_tokens[batch_index*batch_size:(batch_index+1)*batch_size]

    # Extract per-residue representations (on CPU)
    # only the tokens are given the model, 
    # repr_layers only returns the 33 layers as representtion for every amino aids
    # return_contacts predicts contracts between AA

    with torch.no_grad():
        results = model(batch_to_run, repr_layers=[last_layer], return_contacts=True)
    del results["logits"]
    del results["attentions"]


    # Extract the embeddings from the model output (layer 33 for ESM-1b)
    embeddings = results['representations'][last_layer]  # Choose layer 33 for the embeddings

    # Concatenate embeddings for this batch
    all_embeddings.append(embeddings)
    all_contacts.append(results['contacts'])

# Concatenate embeddings from all batches into a single tensor
concatenated_embeddings = torch.cat(all_embeddings, dim=0)
concatenated_contacts = torch.cat(all_contacts, dim=0)



results = {"contacts" : concatenated_contacts, 
           "representations" : {last_layer :concatenated_embeddings}}







# # Generate per-sequence representations via averaging
# # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
# sequence_representations = []
# for i, tokens_len in enumerate(batch_lens):
#     sequence_representations.append(token_representations[i, 1 : tokens_len - 1].mean(0))


# attentions scored for pos to pos
# # Look at the unsupervised self-attention map contact predictions
# import matplotlib.pyplot as plt
# for (_, seq), tokens_len, attention_contacts in zip(data, batch_lens, results["contacts"]):
#     plt.matshow(attention_contacts[: tokens_len, : tokens_len])
#     plt.title(seq)
#     plt.show()




embeding
0.0 % done


In [ ]:

model_name.split("_")

import pickle 
print("saving")

save_file = ""
if file_args =="cova":
    save_file = "cova_"

with open(f"../data/interim/{save_file}embed_EMS_{model_name.split("_")[2]}_{chain_mode}", 'wb') as f:
    pickle.dump(results, f)


print("done")